## 趋势追踪-组合指标择时

In [ ]:
'''
趋势追踪-组合指标择时
作者:一箩筐

简介:计算指数的SMA,MACD,DMA,TRIX指标,当出现以下情况时获得交易信号:
(1)SMA金叉,signal=1;SMA死叉,signal=-1
(2)MACD金叉,signal=1;MACD死叉,signal=-1
(3)DMA上穿AMA,signal=1;DMA下穿AMA,signal=-1;
(4)TRIX上穿MATTRIX,signal=1;TRIX下穿MATTRIX,signal=-1
累计signals>=1时买入;
累计signals<=-1时卖出;


'''

import talib as tl

############################## 以下为主要函数  ################################
#initialize()
#handle_bar_dict()

# 初始化函数 ##################################################################
def init(context):
    set_params(context)                             # 设置策略常量
    set_variables(context)                          # 设置中间变量
    set_backtest(context)                           # 设置回测条件
    
#1.设置策略参数
def set_params(context):
    context.stock = '000001.SH'              # 设置要交易的指数
    context.SMA_s = 4                        # 设置SMA短期均线日期
    context.SMA_l = 40                       # 设置SMA长期均线日期
    context.fastperiod = 12                  # 设置MACD的fastperiod
    context.slowperiod = 26                  # 设置MACD的slowperiod
    context.signalperiod = 9                 # 设置MACD的signalperiod
    context.DMA_S = 4                        # 设置DMA的短期均线日期S
    context.DMA_L = 40                       # 设置DMA的长期均线日期L
    context.DMA_M = 20                       # 设置DMA的均线差值
    context.TRIX_N = 20                      # 设置TRIX的N值
    context.TRIX_M = 60                      # 设置TRIX的M值
#2.设置中间变量
def set_variables(context):
    context.signal = 0                       # 设置信号值(1或0空仓,3买入)
#3.设置回测条件
def set_backtest(context):
    set_benchmark('000001.SH')               # 设置基准
    set_slippage(PriceSlippage(0.002))       # 设置可变滑点
    
    
# 每日开盘执行###################################################################
def handle_bar(context,bar_dict):
    # 计算交易信号
    context.signal = stock_to_signals(context,bar_dict)                     
    # 执行买卖操作
    trade_operation(context.signal,context.stock)

#4.计算SMA信号
def SMA_signal(context,bar_dict):
    value = history(context.stock,['close'],200,'1d',True,'pre')
    value = value.dropna()
    close = value.close.values
    # 计算移动均线值
    sma = tl.SMA(close,context.SMA_s)
    lma = tl.SMA(close,context.SMA_l)
    # 判断信号
    if sma[-1]>sma[-2] and sma[-1]>lma[-1] and sma[-2]<lma[-2]:
        return 1
    elif sma[-1]<sma[-2] and sma[-1]<lma[-1] and sma[-2]>lma[-2]:
        return -1
    else:
        return 0
#5.计算MACD信号
def MACD_signal(context,bar_dict):
    value = history(context.stock,['close'],200,'1d',True,'pre')
    value = value.dropna()
    close = value.close.values
    # 计算macd值
    macd, dif, dea = tl.MACD(close,context.fastperiod,context.slowperiod,context.signalperiod)
    # 判断信号
    if dif[-1]>dif[-2] and dif[-1]>dea[-1] and dif[-2]<dea[-2] and dif[-1]>0:
        return 1
    elif dif[-1]<dif[-2] and dif[-1]<dea[-1] and dif[-2]>dea[-2] and dif[-1]<0:
        return -1
    else:
        return 0
#6.计算DMA信号
def DMA_signal(context,bar_dict):
    value = history(context.stock,['close'],200,'1d',True,'pre')
    value = value.dropna()
    close = value.close.values
    # 计算移动均线值和差值
    sma = tl.SMA(close,context.DMA_S)
    lma = tl.SMA(close,context.DMA_L)
    # 计算DMA
    DMA = sma-lma
    AMA = tl.SMA(DMA,context.DMA_M)
    # 判断信号
    if DMA[-1]>DMA[-2] and DMA[-1]>AMA[-1] and DMA[-2]<AMA[-2]:
        return 1
    elif DMA[-1]<DMA[-2] and DMA[-1]<AMA[-1] and DMA[-2]>AMA[-2]:
        return -1
    else:
        return 0
#7.计算TRIX信号
def TRIX_signal(context,bar_dict):
    value = history(context.stock,['close'],300,'1d',True,'pre')
    value = value.dropna()
    close = value.close.values
    # 计算TR
    EMA1 = tl.SMA(close,context.TRIX_N)
    EMA2 = tl.SMA(EMA1,context.TRIX_N)
    TR = tl.SMA(EMA2,context.TRIX_N)
    # 计算TRIX 和 MATTRIX
    value['TR'] = TR
    value['TRIX'] = value.TR/value.TR.shift(1)-1.0
    TRIX = value.TRIX.values
    MATTRIX = tl.SMA(TRIX,context.TRIX_M)
    # 判断信号
    if TRIX[-1]>TRIX[-2] and TRIX[-1]>MATTRIX[-1] and TRIX[-2]<MATTRIX[-2]:
        return 1
    elif TRIX[-1]<TRIX[-2] and TRIX[-1]<MATTRIX[-1] and TRIX[-2]>MATTRIX[-2]:
        return -1
    else:
        return 0
#8.计算交易信号
def stock_to_signals(context,bar_dict):
    signal_1 = SMA_signal(context,bar_dict)                   #计算SMA信号
    signal_2 = MACD_signal(context,bar_dict)                  #计算MACD信号
    signal_3 = DMA_signal(context,bar_dict)                   #计算DMA信号
    signal_4 = TRIX_signal(context,bar_dict)                  #计算TRIX信号
    #返回信号值
    return signal_1+signal_2+signal_3+signal_4
#9.执行买卖操作
def trade_operation(signal,stock):
    if signal>=1:
        order_target_percent(stock,1)
    if signal<=-1:
        order_target_percent(stock,0)